In [1]:
#Implementing the Self-Attention Mechanism from Scratch in PyTorch!


hello


```python
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the Attention module
class Attention(nn.Module):
    """
    This class implements a self-attention mechanism, which allows the model to
    attend to different parts of the input sequence based on their relevance.
    """
    def __init__(self, d_in, d_out):
        """
        Initializes the Attention module.

        Args:
            d_in: Dimensionality of the input tensor (number of features).
            d_out: Dimensionality of the output tensor (number of attention heads).
        """
        super().__init__()
        self.d_in = d_in
        self.d_out = d_out

        # Define linear transformations for Keys, Queries, and Values
        # These transformations project the input tensor into different feature spaces.
        self.Q = nn.Linear(d_in, d_out)  # Query projection
        self.K = nn.Linear(d_in, d_out)  # Key projection
        self.V = nn.Linear(d_in, d_out)  # Value projection

    def forward(self, x):
        """
        Performs the forward pass of the attention module.

        Args:
            x: Input tensor of shape (batch_size, seq_len, d_in).

        Returns:
            Output tensor of shape (batch_size, seq_len, d_out), representing the
            weighted average of the values based on the attention weights.
        """
        # Project the input tensor into Keys, Queries, and Values
        queries = self.Q(x)  # (batch_size, seq_len, d_out)
        keys = self.K(x)    # (batch_size, seq_len, d_out)
        values = self.V(x)  # (batch_size, seq_len, d_out)

        # Calculate the interaction matrix between Keys and Queries
        scores = torch.bmm(queries, keys.transpose(1, 2)) # (batch_size, seq_len, seq_len)
        scores = scores / (self.d_out ** 0.5)  # Scale scores for numerical stability

        # Apply softmax to the scores to obtain attention weights
        attention = F.softmax(scores, dim=2)  # (batch_size, seq_len, seq_len)

        # Compute the weighted average of the Values based on the attention weights
        hidden_states = torch.bmm(attention, values)  # (batch_size, seq_len, d_out)

        return hidden_states

# Example usage with MNIST dataset
if __name__ == "__main__":
    # Assuming you have the MNIST dataset loaded (e.g., using torchvision)
    # ...
    # Replace with your MNIST dataset loading code
    # ...

    # Create an instance of the Attention module
    attention_layer = Attention(d_in=28*28, d_out=128)  # Example: 28x28 image size

    # Example usage with a batch of MNIST images
    # images: Tensor of shape (batch_size, 1, 28, 28)
    # Flatten the images for the Attention module
    flattened_images = images.view(batch_size, -1)  # (batch_size, 28*28)

    # Calculate the attention-weighted hidden states
    attention_output = attention_layer(flattened_images)
    # ...
    # Use the attention_output for further processing or classification
    # ...
```

**Explanation:**

**1. Initialization:**
   - The `__init__` method sets up the linear transformations for Keys, Queries, and Values. The dimensionality of the input (`d_in`) and output (`d_out`) are specified.

**2. Forward Pass:**
   - `forward(x)` performs the core attention computation:
     - **Projection:** The input tensor `x` is projected into the Key, Query, and Value spaces using the linear transformations (`self.Q`, `self.K`, `self.V`).
     - **Interaction:** The dot product of Queries and Keys (after scaling) is computed to generate an interaction matrix called `scores`. This matrix represents the relevance of each input element to other elements in the sequence.
     - **Softmax:** Softmax is applied to the `scores` to obtain attention weights, which sum up to 1 for each input element. These weights indicate how much attention is given to each element in the sequence.
     - **Weighted Average:** The attention weights are used to compute a weighted average of the Values, resulting in a new representation called `hidden_states`. This representation captures the context-aware information from the input sequence.

**3. MNIST Example:**
   - The example demonstrates how to use the `Attention` layer with the MNIST dataset.
   - The image is first flattened into a vector.
   - The `attention_layer` is applied to the flattened image to obtain the attention-weighted hidden states.
   - You can use these hidden states for further processing or classification tasks.

**Key Points:**

- The `Attention` module is a flexible building block that can be integrated into various deep learning models.
- It helps the model to focus on relevant parts of the input sequence, leading to improved performance in tasks like machine translation, text summarization, and image captioning.
- The scaling factor (`self.d_out ** 0.5`) helps to stabilize numerical computation by preventing large values from dominating the softmax calculation.
- In the MNIST example, the attention mechanism is applied to the flattened image, but it can be adapted to work with different input formats, such as sequences of words or time series data.

This code provides a basic implementation of self-attention. More sophisticated attention mechanisms, such as multi-head attention and transformer architectures, build upon these fundamental concepts.


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define the Attention module
class Attention(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.d_in = d_in
        self.d_out = d_out

        self.Q = nn.Linear(d_in, d_out)
        self.K = nn.Linear(d_in, d_out)
        self.V = nn.Linear(d_in, d_out)

    def forward(self, x):
        queries = self.Q(x)
        keys = self.K(x)
        values = self.V(x)

        scores = torch.bmm(queries, keys.transpose(1, 2))
        scores = scores / (self.d_out ** 0.5)

        attention = F.softmax(scores, dim=2)
        hidden_states = torch.bmm(attention, values)

        return hidden_states

# Define the MNIST classifier model
class MNISTClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.attention = Attention(d_in=64*7*7, d_out=128)
        self.fc1 = nn.Linear(128, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten the feature map
        x = x.view(x.size(0), 1, -1)  # Reshape for attention
        x = self.attention(x)
        x = x.view(x.size(0), -1)  # Flatten after attention
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define the training parameters
batch_size = 64
epochs = 10
learning_rate = 0.001

# Load the MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model, loss function, and optimizer
model = MNISTClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

# Evaluate the model on the test set
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print('Accuracy on test set: {:.2f}%'.format(100. * correct / total))


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.299715
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.274164
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.197592
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.119946
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.099907
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.040582
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.108120
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.101416
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.060152
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.008438
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.122799
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.109486
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.043232
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.070252
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.028653
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.118166
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.046321
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.020224
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.061158
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.031930
Train Epoch:



**Explanation:**

1. **Import Libraries:**
   - `torch`: PyTorch for deep learning operations.
   - `torch.nn`: PyTorch's neural network module.
   - `torch.nn.functional`: PyTorch's functional API for activation functions, etc.
   - `torchvision.datasets`: Datasets like MNIST.
   - `torchvision.transforms`: For image transformations.
   - `torch.utils.data.DataLoader`: For loading and batching data.

2. **Attention Module:**
   - The `Attention` class remains the same as before.

3. **MNIST Classifier Model:**
   - `MNISTClassifier` defines a simple convolutional neural network (CNN) for MNIST classification:
     - Two convolutional layers with ReLU activation.
     - Max pooling to downsample feature maps.
     - The `Attention` layer is applied after the convolutional layers to focus on relevant features.
     - Two fully connected (FC) layers for classification.

4. **Dataset Loading:**
   - The MNIST dataset is loaded using `torchvision.datasets.MNIST`.
   - `DataLoader` is used to create batches for training and testing.

5. **Training:**
   - The training loop iterates over epochs and batches.
   - The model makes predictions using `model(data)`.
   - The loss is calculated using `criterion(output, target)`.
   - Backpropagation and optimization are performed using `loss.backward()` and `optimizer.step()`.
   - Training progress is printed every 100 batches.

6. **Evaluation:**
   - The model is evaluated on the test set using `torch.no_grad()` to disable gradient calculations.
   - Accuracy is calculated and printed.

**Key Points:**

- This code demonstrates how to integrate the self-attention mechanism into a CNN model for MNIST classification.
- The attention layer helps the model focus on relevant features in the image, potentially improving classification accuracy.
- The code provides a basic example. You can experiment with different architectures, hyperparameters, and attention mechanisms to further enhance the model's performance.

This code should provide a clear and complete example of how to use the self-attention module with the MNIST dataset.
